# Pattern Generation

## Turing Pattern Generated by Turing Model

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
import csv
import os
from noise import pnoise2
import uuid

# Define Turing Model
class TuringModel:
    def __init__(self, length=500, dt=0.5, T=1000, 
                 a_u=0.08, b_u=-0.08, c_u=0.04, d_u=0.03, D_u=0.02, F_max=0.2, 
                 a_v=0.1, b_v=0.0, c_v=-0.15, d_v=0.08, D_v=0.5, G_max=0.5, 
                 initial_u=None, initial_v=None):
        self.length = length
        self.dt = dt
        self.T = T
        self.a_u = a_u
        self.b_u = b_u
        self.c_u = c_u
        self.d_u = d_u
        self.D_u = D_u
        self.F_max = F_max
        self.a_v = a_v
        self.b_v = b_v
        self.c_v = c_v
        self.d_v = d_v
        self.D_v = D_v
        self.G_max = G_max
        
        self.u = initial_u if initial_u is not None else np.random.rand(length, length)
        self.v = initial_v if initial_v is not None else np.random.rand(length, length)

    def laplacian(self, arr):
        return (
            np.roll(arr, 1, axis=0) + np.roll(arr, -1, axis=0) +
            np.roll(arr, 1, axis=1) + np.roll(arr, -1, axis=1) - 
            4 * arr
        )

    def F_func(self, u, v):
        return self.a_u * u + self.b_u * v + self.c_u

    def G_func(self, u, v):
        return self.a_v * u + self.b_v * v + self.c_v

    def run(self):
        for _ in range(self.T):
            F_val = self.F_func(self.u, self.v)
            G_val = self.G_func(self.u, self.v)
            F_val = np.clip(F_val, 0, self.F_max)
            G_val = np.clip(G_val, 0, self.G_max)
            
            self.u += self.dt * (F_val - self.d_u * self.u + self.D_u * self.laplacian(self.u))
            self.v += self.dt * (G_val - self.d_v * self.v + self.D_v * self.laplacian(self.v))

    def save_image(self, file_path, title=""):
        self.run()
        fig, axs = plt.subplots(1, 1, figsize=(10, 10))
        im_v = axs.imshow(self.v, cmap='coolwarm')

        axs.set_xticks([])
        axs.set_yticks([])

        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
        
        plt.savefig(file_path, bbox_inches='tight', pad_inches=0)
        
        plt.close()


/var/folders/wz/k8zp21d91q301dlw5467v5sm0000gn/T/ipykernel_23062/2332188979.py:162: RuntimeWarning: divide by zero encountered in scalar divide
  invDenom = 1 / (dot00 * dot11 - dot01 * dot01)
/var/folders/wz/k8zp21d91q301dlw5467v5sm0000gn/T/ipykernel_23062/2332188979.py:163: RuntimeWarning: invalid value encountered in scalar multiply
  u = (dot11 * dot02 - dot01 * dot12) * invDenom
/var/folders/wz/k8zp21d91q301dlw5467v5sm0000gn/T/ipykernel_23062/2332188979.py:164: RuntimeWarning: invalid value encountered in scalar multiply
  v = (dot00 * dot12 - dot01 * dot02) * invDenom


In [ ]:
# Select parameters randomly
def generate_param_sets(varying_ranges, fixed_params, num_samples):
    keys = varying_ranges.keys()
    for _ in range(num_samples):
        params = {key: np.random.uniform(low, high) for key, (low, high) in varying_ranges.items()}
        params.update(fixed_params)
        # print(params)
        yield params

# Generate initial pattern randomly
def generate_random_pattern(length, pattern_type):
    pattern = np.zeros((length, length))
    if pattern_type == 'sine_wave':
        x = np.linspace(0, 2 * np.pi, length)
        y = np.linspace(0, 2 * np.pi, length)
        xv, yv = np.meshgrid(x, y)
        pattern = np.sin(xv * 10 + yv * 10)
    elif pattern_type == 'perlin_noise':
        scale = 100.0
        octaves = 6
        persistence = 0.5
        lacunarity = 2.0
        for x in range(length):
            for y in range(length):
                pattern[x][y] = pnoise2(x / scale, 
                                        y / scale, 
                                        octaves=octaves, 
                                        persistence=persistence, 
                                        lacunarity=lacunarity, 
                                        repeatx=length, 
                                        repeaty=length, 
                                        base=42)
    elif pattern_type == 'grid':
        spacing = np.random.randint(5, 20)
        pattern[::spacing, ::spacing] = 1
    elif pattern_type == 'circles':
        x0, y0 = np.random.randint(0, length, 2)
        radius = np.random.randint(2, length // 4)
        for x in range(length):
            for y in range(length):
                if ((x - x0) ** 2 + (y - y0) ** 2) < radius ** 2:
                    pattern[x, y] = 1
    elif pattern_type == 'stripes':
        spacing = np.random.randint(5, 20)
        pattern[:, ::spacing] = 1
    elif pattern_type == 'checkerboard':
        spacing = np.random.randint(2, 10)
        pattern[::spacing, ::spacing] = 1
        pattern[1::spacing, 1::spacing] = 1
    elif pattern_type == 'gaussian_spots':
        num_spots = np.random.randint(5, 20)
        for _ in range(num_spots):
            x0, y0 = np.random.randint(0, length, 2)
            sigma = np.random.randint(length // 20, length // 10)
            x = np.arange(0, length)
            y = np.arange(0, length)
            xv, yv = np.meshgrid(x, y)
            d = np.sqrt((xv - x0) ** 2 + (yv - y0) ** 2)
            spots = np.exp(-(d ** 2 / (2.0 * sigma ** 2)))
            spots = np.clip(spots, 0, 1)
            pattern += spots
    elif pattern_type == 'spirals':
        x = np.linspace(-np.pi, np.pi, length)
        y = np.linspace(-np.pi, np.pi, length)
        xv, yv = np.meshgrid(x, y)
        pattern = np.sin(np.sqrt(xv ** 2 + yv ** 2) * 10 + np.arctan2(yv, xv) * 10)
        pattern = np.clip(pattern, 0, 1)
    elif pattern_type == 'random_noise':
        pattern = np.random.rand(length, length)
    elif pattern_type == 'ellipses':
        x0, y0 = np.random.randint(0, length, 2)
        a = np.random.randint(5, length // 4)
        b = np.random.randint(5, length // 4)
        for x in range(length):
            for y in range(length):
                if ((x - x0) ** 2 / a ** 2 + (y - y0) ** 2 / b ** 2) <= 1:
                    pattern[x, y] = 1
    elif pattern_type == 'triangles':
        x0, y0 = np.random.randint(0, length // 2, 2)
        x1, y1 = np.random.randint(length // 2, length, 2)
        x2, y2 = np.random.randint(0, length, 2)
        triangle = np.array([[x0, y0], [x1, y1], [x2, y2]])
        for x in range(length):
            for y in range(length):
                p = np.array([x, y])
                v0 = triangle[2] - triangle[0]
                v1 = triangle[1] - triangle[0]
                v2 = p - triangle[0]
                dot00 = np.dot(v0, v0)
                dot01 = np.dot(v0, v1)
                dot02 = np.dot(v0, v2)
                dot11 = np.dot(v1, v1)
                dot12 = np.dot(v1, v2)
                invDenom = 1 / (dot00 * dot11 - dot01 * dot01)
                u = (dot11 * dot02 - dot01 * dot12) * invDenom
                v = (dot00 * dot12 - dot01 * dot02) * invDenom
                if (u >= 0) and (v >= 0) and (u + v <= 1):
                    pattern[x, y] = 1
    else:
        raise ValueError(f"No pattern_type called {pattern_type}")
    return pattern

def generate_initial_conditions(length, pattern_types, num_conditions, max_patterns=3):
    initial_conditions = []
    for _ in range(num_conditions):
        u_patterns = np.random.choice(pattern_types, np.random.randint(1, max_patterns+1), replace=False)
        v_patterns = np.random.choice(pattern_types, np.random.randint(1, max_patterns+1), replace=False)
        initial_u = np.zeros((length, length))
        initial_v = np.zeros((length, length))
        for pattern_type in u_patterns:
            initial_u += generate_random_pattern(length, pattern_type)
        for pattern_type in v_patterns:
            initial_v += generate_random_pattern(length, pattern_type)
        initial_u = initial_u / len(u_patterns)
        initial_v = initial_v / len(v_patterns)
        initial_conditions.append((initial_u, initial_v, u_patterns, v_patterns))
    return initial_conditions

# Simulate pattern formation
def generate_and_save_patterns(varying_ranges, fixed_params, base_path, pattern_types=['sine_wave'], num_images=100):
    num_initial_conditions = num_images
    initial_conditions = generate_initial_conditions(fixed_params['length'], pattern_types, num_initial_conditions, max_patterns=len(pattern_types))
    # initial_conditions = generate_initial_conditions(fixed_params['length'], pattern_types, num_initial_conditions, max_patterns=2)
    param_sets = generate_param_sets(varying_ranges, fixed_params, num_images)
    idx = 0
    metadata_file = os.path.join(base_path, "metadata.csv")
    
    with open(metadata_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        if os.stat(metadata_file).st_size == 0:
            writer.writerow(['filename', 'a_u', 'b_u', 'c_u', 'd_u', 'D_u', 'F_max', 
                             'a_v', 'b_v', 'c_v', 'd_v', 'D_v', 'G_max', 'u_patterns', 'v_patterns'])
        
        for initial_u, initial_v, u_patterns, v_patterns in initial_conditions:
            params = next(param_sets)
            model = TuringModel(initial_u=initial_u, initial_v=initial_v, **params)
            title = ", ".join([f"{key}={value:.3f}" for key, value in params.items()])
            file_uuid = str(uuid.uuid4())
            file_path = f"{base_path}/pattern_{file_uuid}.png"
            model.save_image(file_path, title=title)
            
            writer.writerow([file_path] + list(params.values()) + [u_patterns, v_patterns])
            
            idx += 1
            if idx >= num_images:
                return


In [ ]:
fixed_params = {
    "length": 100,
    "dt": 0.05,
    "T": 10000,

    # "d_u": 0.03,
    # "D_u": 0.02,
    # "a_u": 0.08,
    # "b_u": -0.08,
    # "c_u": 0.03,
    # "F_max": 0.2,

    # "d_v": 0.08,
    # "D_v": 0.5,
    # "a_v": 0.1,
    # "b_v": 0.,
    # "c_v": -0.15,
    # "G_max": 0.5

}

varying_ranges = {
    "d_u": (0.01, 0.05),
    "D_u": (0, 0.04),
    "a_u": (0.06, 0.1),
    "b_u": (-0.1, -0.06),
    "c_u": (0.01, 0.05),
    "F_max": (0.18, 0.22),

    "d_v": (0.06,0.1),
    "D_v": (0.48, 0.52),
    "a_v": (0.08, 0.12),
    "b_v": (-0.02, 0.02),
    "c_v": (-0.17, -0.13),
    "G_max": (0.48, 0.52)
}


# Combinations of initial conditions
pattern_types1 = ['sine_wave', 'perlin_noise', 'grid', 'circles', 'stripes', 'checkerboard', 'gaussian_spots', 'spirals', 'random_noise', 'ellipses', 'triangles']
pattern_types2 = ['random_noise']

os.makedirs("./tmp_turing_dataset", exist_ok=True)
# os.makedirs("./tmp", exist_ok=True)

# Generate and save patterns with different initial conditions and parameters
generate_and_save_patterns(varying_ranges, fixed_params, "./tmp_turing_dataset", pattern_types=pattern_types1, num_images=1024)
generate_and_save_patterns(varying_ranges, fixed_params, "./tmp_turing_dataset", pattern_types=pattern_types2, num_images=1024)
